In [1]:
import os

In [6]:
user = os.getenv("user")
warehouse = os.getenv("warehouse")
schema= os.getenv("schema")
database = os.getenv("database")
role =  os.getenv("role")
account =  os.getenv("account")
password= os.getenv("password")

In [7]:
from snowflake.snowpark.session import Session

ModuleNotFoundError: No module named 'cryptography.__about__'

In [8]:
!pip install cryptography
!pip install paramiko

     |████████████████████████████████| 3.8MB 3.2MB/s eta 0:00:01
     |████████████████████████████████| 450kB 83.8MB/s eta 0:00:01
     |████████████████████████████████| 122kB 75.4MB/s eta 0:00:01
ERROR: snowflake-connector-python 3.9.0 has requirement urllib3<2.0.0,>=1.21.1; python_version < "3.10", but you'll have urllib3 2.2.2 which is incompatible.
ERROR: snowflake-snowpark-python 1.0.0 has requirement cloudpickle<=2.0.0,>=1.6.0, but you'll have cloudpickle 3.0.0 which is incompatible.
ERROR: snowflake-snowpark-python 1.0.0 has requirement snowflake-connector-python<3.0.0,>=2.7.12, but you'll have snowflake-connector-python 3.9.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement cryptography<39.0.0,>=3.1.0, but you'll have cryptography 42.0.8 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement packaging<24,>=20.9, but you'll have packaging 24.1 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement pandas<2,>=1.0.0, but 